In [ ]:
import pandas as pd
import numpy as np
import qiyuanpublic as qyp
from datetime import datetime

In [ ]:
filename = "运营指标_%s.xlsx" % datetime.strftime(datetime.now().date(), "%Y-%m-%d")
writer = pd.ExcelWriter(filename)

In [ ]:

sql = """
-- 组合
select DATE(a.created_at) "apply_date",
CONCAT(year(a.created_at),"-Q",QUARTER(a.created_at)) "Quarter",
CONCAT(year(a.created_at),"-M",month(a.created_at)) "Month",
CONCAT(year(a.created_at),"-W",week(a.created_at)) "Week",
if(a.product_name = "CashNiJuan","CNJ","P2G&Jet") as product_name,
a.order_no,a.order_type,a.customer_phone_num "phone",a.application_status,
if(a.application_status in ("CreateApplication", "InitialAuditingReject", "AutoAuditing", "AutoReject", "RiskControlReject"), null, 1) "JSTG",
if(a.application_status in ("AuditingPassed"), 1, null) "RSTG",
 b.loan_flow_status,b.loan_principal,c.actual_paid_off_date,c.due_date,c.FDQ,
d.DPD3,e.DPD7
from phl_approval_prod.approval_applications a
LEFT JOIN
(select order_no, loan_principal, loan_flow_status from phl_core_prod.core_loans where loan_flow_status="LoanSuccess") b
on a.order_no=b.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
if(if(actual_paid_off_date is null,DATEDIFF(CURDATE(),due_date),DATEDIFF(actual_paid_off_date,due_date))>0,1,0) "FDQ"
from phl_core_prod.core_repayment_schedules where extend_period=0
and due_date<=DATE_SUB(CURDATE(),INTERVAL 1 day)) c
on b.order_no = c.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
if(if(actual_paid_off_date is null,DATEDIFF(CURDATE(),due_date),DATEDIFF(actual_paid_off_date,due_date))>3,1,0) "DPD3"
from phl_core_prod.core_repayment_schedules where  extend_period=0
and due_date<=DATE_SUB(CURDATE(),INTERVAL 3 day)) d
on b.order_no = d.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
if(if(actual_paid_off_date is null,DATEDIFF(CURDATE(),due_date),DATEDIFF(actual_paid_off_date,due_date))>7,1,0) "DPD7"
from phl_core_prod.core_repayment_schedules where (product_name = "Peso2Go" or product_name = "JetPeso") and extend_period=0
and due_date<=DATE_SUB(CURDATE(),INTERVAL 7 day)) e
on b.order_no = e.order_no
where  a.deleted_at is null and a.created_at>DATE_SUB(CURDATE(),INTERVAL 60 day)
"""
df = qyp.sql_to_df(sql, db_info=qyp.get_DB()[2])
df["新老标签"] = df["order_type"].map(lambda s: "新客" if s in ["RepeatApplicationOrder", "NewApplicationOrder"] else "老客")
df_result = df.groupby(["apply_date", "新老标签", "product_name"]).agg(
    {"order_no": "count", "JSTG": "sum",  "RSTG": "sum", "loan_flow_status": "count",
     "loan_principal": "sum", "FDQ": "sum", "DPD7": "sum"})
df_result["机审通过率"] = df_result["JSTG"]/df_result["order_no"]
df_result["人审通过率"] = df_result["RSTG"]/df_result["JSTG"]
df_result["放款通过率"] = df_result["loan_flow_status"]/df_result["order_no"]
df_result["FDQ_%"] = df_result["FDQ"]/df_result["loan_flow_status"]
df_result["DPD7_%"] = df_result["DPD7"]/df_result["loan_flow_status"]
df_result.rename(columns={"order_no": "申请笔数", "loan_flow_status": "放款笔数", "loan_principal": "放款金额"}, inplace=True)
df_result = df_result[["申请笔数", "机审通过率", "人审通过率", "放款笔数", "放款通过率", "放款金额", "FDQ", "FDQ_%", "DPD7", "DPD7_%"]].reset_index()
# print(df_result)
for name in ["P2G&Jet", "CNJ"]:
    for type in ["新客", "老客"]:
        df_r = df_result.loc[(df_result["product_name"] == name) & (df_result["新老标签"] == type)]
        df_r.reset_index(drop=True, inplace=True)
        df_r.drop(columns=["新老标签", "product_name"], axis=1, inplace=True)
        if len(df_r):
            df_r.to_excel(writer, sheet_name="%s_%s" % (name, type))
# df_result.to_excel("运营指标.xlsx")
writer.save()